In [1]:
import os
from tkinter import *
from tkinter import filedialog
import tkinter as tk
import PIL.Image
from PIL import Image, ImageTk
from PIL import ImageDraw # для отрисовки 68 landmarks

In [2]:
import numpy as np
import dlib
import cv2
from matplotlib import pyplot as plt
import shutil # для копирование файлов
import imgaug # для аугментации
from imgaug import augmenters as iaa # для аугментации

# Сериализация

In [3]:
import pickle

class Serializer_DB:
    
    name_of_db = 'db_persons.pkl'
    
    # сохранение данных в файле
    # known_face_encodings - кодировки лиц (128d векторы)
    # known_face_names - именя людей в соответсвии к кодировкам
    # data_labels - номера людей для классификации
    @staticmethod
    def save_data_to_file(known_face_encodings, known_face_names, data_labels):
    
        db_for_serialize = { "encodings" : known_face_encodings,
                             "names" : known_face_names,
                             "labels" : data_labels}
        print("Writing DB to file...")
        pickle.dump(db_for_serialize, open(Serializer_DB.name_of_db, 'wb'))

    # получение данных из файла
    @staticmethod
    def read_data_from_file():
        print("Reading DB from file...")
        db_from_serialize = pickle.load(open(Serializer_DB.name_of_db, 'rb'))
        return db_from_serialize

# Нормализация фотографии

In [9]:
from collections import OrderedDict

# класс описывающий нормализацию изображения лица
class FaceNormalizer:
    
    FACIAL_LANDMARKS_5_POINTS = OrderedDict([
    ("right_eye", (2, 3)),
    ("left_eye", (0, 1)),
    ("nose", (4))
    ])
    
    def __init__(self, predictor, desiredLeftEye=(0.33, 0.33),
        desiredFaceWidth=256, desiredFaceHeight=None):
        # сохранение определителя ключевых точек лица, желаемая позиция левого глаза
        # и желаемая ширина и высота лица на выходящем изображении
        self.predictor_landmarks = predictor
        self.desiredLeftEye = desiredLeftEye
        self.desiredFaceWidth = desiredFaceWidth
        self.desiredFaceHeight = desiredFaceHeight
        # if the desired face height is None, set it to be the
        # desired face width (normal behavior)
        if self.desiredFaceHeight is None:
            self.desiredFaceHeight = self.desiredFaceWidth
            
    # функция для нормализации лица
    # image - изображение в RGB 
    # gray - серое изображение
    # rect - область лица, выделенная HOG
    def align(self, image, gray, rect):
        # получение 5 ориентиров-landmarks
        shape = self.predictor_landmarks(image, rect)
        # преобразование dlib object в np.array
        shape = self.shape_to_np(shape)
        
        # извлечение левого и правого глаз (x, y)-coordinates
        (lStart, lEnd) = self.FACIAL_LANDMARKS_5_POINTS["left_eye"]
        (rStart, rEnd) = self.FACIAL_LANDMARKS_5_POINTS["right_eye"]
        # извлечение точек левого глаза и правого (x, y)-coordinates
        leftEyePts = shape[lStart:lEnd + 1]
        rightEyePts = shape[rStart:rEnd + 1]
        
        # расчет центра для каждого глаза
        leftEyeCenter = leftEyePts.mean(axis=0).astype("int")
        rightEyeCenter = rightEyePts.mean(axis=0).astype("int")
        # расчет угла между центроидами глаз
        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX)) - 180
        #print(angle)
        
        # вычисление х-коорд правого глаза основанной на x-коорд левого глаза
        desiredRightEyeX = 1.0 - self.desiredLeftEye[0]
        # определение масштаба результирующего изображения, взяв
        # отношение расстояния между глазами в текущем изображении
        # к отношению расстояния глаз в желаемом изображении
        dist = np.sqrt((dX ** 2) + (dY ** 2)) # Евклидово расстояние
        desiredDist = (desiredRightEyeX - self.desiredLeftEye[0]) # 
        desiredDist *= self.desiredFaceWidth
        scale = desiredDist / dist
        
        # вычисление центра между глазами (x, y)-coordinates
        # для вращения фотографии вокруг этого центра
        eyesCenter = ((leftEyeCenter[0] + rightEyeCenter[0]) / 2,
        (leftEyeCenter[1] + rightEyeCenter[1]) / 2)
        #print("eyesCenter = " + str(eyesCenter))
        
        # получение матрицы для поворота и масштабирования лица
        M = cv2.getRotationMatrix2D(center=eyesCenter, angle=angle, scale=scale)
        
        # обновление компонентов матрицы на смещение
        tX = self.desiredFaceWidth * 0.5
        tY = self.desiredFaceHeight * self.desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])
        
        # применение аффинного преобразования
        (w, h) = (self.desiredFaceWidth, self.desiredFaceHeight)
        output = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)
        
        # возвращение нормализованного лица
        return output

    # dlib object -> np.array
    def shape_to_np(self, shape, dtype="int"):
        # initialize the list of (x, y)-coordinates
        coords = np.zeros((shape.num_parts, 2), dtype=dtype)

        # loop over all facial landmarks and convert them
        # to a 2-tuple of (x, y)-coordinates
        for i in range(0, shape.num_parts):
            coords[i] = (shape.part(i).x, shape.part(i).y)

        # return the list of (x, y)-coordinates
        return coords


In [5]:
face_normalizer = FaceNormalizer(landmarks_predictor, desiredFaceWidth=150, desiredFaceHeight=150)

check_img = load_image("klava_koka_ii.jpg")
gray_image = cv2.cvtColor(check_img, cv2.COLOR_BGR2GRAY)
detected_faces = detector(check_img, 1)

normalizedFace = face_normalizer.align(check_img, gray_image, detected_faces[0])

# plt.imshow(normalizedFace)

# face_chip = dlib.get_face_chip(check_img, landmarks_predictor(check_img, detected_faces[0]))
# plt.imshow(face_chip)

NameError: name 'landmarks_predictor' is not defined

# Обработчик лица

In [11]:
# Класс, который выполняет определение области лица и извлечение из нее вектора признаков
class FaceHandler:
    
    # определение моделей
    face_encoder = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")
    detector = dlib.get_frontal_face_detector()  # определение области лица с помощью HOG
    landmarks_predictor = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')
    
    # определение экземпляра нормализатора лица
    face_normalizer = FaceNormalizer(landmarks_predictor, desiredFaceWidth=150, desiredFaceHeight=150)
    
    # Функция для определения лица и возвращение вектора 128d
    def detect_face_and_encode(self, path_to_img, img_name, image_identify=False):

        if image_identify == False:
            img_to_recognize = Utilities.load_image(path_to_img + "/" + img_name)
        else:
            img_to_recognize = Utilities.load_image(path_to_img)

        detected_faces = self.detector(img_to_recognize, 1) # определение области лица HOG

        if len(detected_faces) == 0:
            print("Error: There is no faces in image!")
            return []

        # получение нормализованного лица
        gray_image = cv2.cvtColor(img_to_recognize, cv2.COLOR_BGR2GRAY)
        normalizedFace = self.face_normalizer.align(img_to_recognize, gray_image, detected_faces[0])
        # получение 128d вектора embedding
        face_vector = np.array(self.face_encoder.compute_face_descriptor(normalizedFace, num_jitters=1))
        return face_vector

# Создание базы данных

In [7]:
# класс описывающий базу данных
class DataBase:
    
    known_face_encodings = [] # кодировки лиц из 128d векторов
    known_face_names = [] # соответствующие имена векторам
    data_labels = [] # соответствующие метки
    
    def create_db(self, face_handler):

        # папка с эталонными фото
        root_dir = "images_db"
        
        # удаление скрытого файла
        if os.path.exists(root_dir + "/.DS_Store"):
            os.remove(root_dir + "/.DS_Store")
            
        count_persons = len(os.listdir(root_dir)) # кол-во человек в БД

        id_label = 0
        for person_dir in os.listdir(root_dir):
            print("Process person: " + person_dir + " - " + str(id_label + 1) + "/" + str(count_persons))

            path_to_person = root_dir + "/" + person_dir

            # удаление скрытого файла
            if os.path.exists(path_to_person + "/.DS_Store"):
                os.remove(path_to_person + "/.DS_Store")

            imgs_in_person_dir = os.listdir(path_to_person)

            # если в папке человека 1 фото, то нужна аугментация
            if len(imgs_in_person_dir) == 1:
                print("Process of augmentation...")
                Augmentator.augmentate_image(path_to_person + "/" + imgs_in_person_dir[0])

            # обновить список фотографий после аугментации
            imgs_in_person_dir = os.listdir(path_to_person)

            # проход по всем фото в папке одного человека
            for image in imgs_in_person_dir:
                print(image)

                # определение области лица и получение закодированного лица
                encoded_face = face_handler.detect_face_and_encode(path_to_person, image)
                # добавление закодированного лица в БД
                if len(encoded_face) == 128:
                    self.add_to_db(encoded_face, id_label, person_dir)

            id_label += 1 # изменения номера отметки для следующего человека
        
        print(self.known_face_names)
    
    #функция для добавления вектора лица в БД
    def add_to_db(self, vector_128d, id_num, person_name):

        self.known_face_encodings.append(vector_128d)
        self.known_face_names.append(person_name)
        self.data_labels.append(id_num)
    

In [12]:
data_base = DataBase()

face_handler = FaceHandler()

# если нужна сохраненная БД
need_new_DB = False

if need_new_DB == False:
    # используется сохраненная база
    data_from_existing_DB = Serializer_DB.read_data_from_file()
    data_base.known_face_encodings = data_from_existing_DB["encodings"]
    data_base.known_face_names = data_from_existing_DB["names"]
    data_base.data_labels = data_from_existing_DB["labels"]
else:
    # Вызов функции для создания БД: флаг False - если аугментация не нужна
    data_base.create_db(face_handler)
    # сериализация данных в файл
    Serializer_DB.save_data_to_file(data_base.known_face_encodings, data_base.known_face_names, data_base.data_labels)

Reading DB from file...


# Аугментация фотографии

In [6]:
# класс, отвечающий за расширение обучающего набора эталонного изображения
class Augmentator:
    
    @staticmethod
    def augmentate_image(path_to_image):

        # получение пути к фото и названия фото
        head_tail = os.path.split(path_to_image)
        path_dir = head_tail[0]
        img_name = head_tail[1]

        # загрузка изображения
        img = Utilities.load_image(path_to_image)

        flag_flip = 1.0
        brightness_dark = -10
        brightness_light = 10

        for i in range(6):
            aug_pipeline = iaa.Sequential([

            iaa.Affine(scale=(0.8, 1.1)),
            iaa.Crop(percent=(0, 0.15)),
            iaa.Fliplr(flag_flip),
            iaa.AddToBrightness((brightness_dark, brightness_light))
            ],
            random_order=True)

            augmented_photo = np.array(aug_pipeline.augment_image(img))

            # сохранения аугментированного изображения
            new_img_name = path_dir + "/" + str(i + 1) + ".jpg"
            cv2.imwrite(new_img_name, cv2.cvtColor(augmented_photo, cv2.COLOR_RGB2BGR))

            # изменение параметров
            flag_flip = 0.0 if flag_flip == 1.0 else 1.0
            brightness_dark -= 20
        #     brightness_light += 10

In [9]:
#augmentate_image("images_db/Aaron_Piersol/Aaron_Piersol.jpg")

# KNN классификатор

In [5]:
# KNN алгоримт
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

class Classifier:
    
    def __init__(self):
        self.X_train = []
        self.X_test = []
        self.y_train = []
        self.y_test = []
        
        self.classifier = None
    
    # метод для обучения классификатора
    # data - данные изображения людей базы данных
    def train_classifier_KNN(self, data):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(data.known_face_encodings, 
                                                                                data.data_labels, 
                                                                                test_size=0.3)
        
        self.classifier = KNeighborsClassifier(n_neighbors = 3, metric=self.my_dist)
        self.classifier.fit(self.X_train, self.y_train)
        
        print("Re-FIT KNN!")

    def my_dist(self, x, y):
    
        diff = np.linalg.norm(x - y)
        if diff > 0.6:
    #         print("ZERO")
            return 1
        else:
    #         print("GOOD")
            return diff

In [13]:
my_classifier = Classifier()

my_classifier.train_classifier_KNN(data_base)

yhat_class = my_classifier.classifier.predict(my_classifier.X_test)
yhat_class

Re-FIT KNN!


array([ 5,  6, 10,  7,  2,  2,  0,  4,  2,  8,  0,  3,  0,  2,  9,  9,  8,
        9,  9,  4,  7,  3,  4,  7])

In [14]:
my_classifier.y_test

[5, 6, 10, 7, 2, 2, 0, 4, 2, 8, 0, 3, 0, 2, 9, 9, 8, 9, 9, 4, 7, 3, 4, 7]

# Вспомогательные функции

In [4]:
# класс, содержащий вспомогательные функции
class Utilities:
    
    # преобразование массива known_name, где повторяются имена людей
    @staticmethod
    def get_unique_values(values):
        uniques = []
        for v in values:
            if v not in uniques:
                uniques.append(v)
        return uniques
    
    # функция для загрузки фотографии
    @staticmethod
    def load_image(path_to_image):
        img = PIL.Image.open(path_to_image)
        
        return np.array(img)
    
    @staticmethod    
    def rect_to_box(rect):
        # преобразование квадрата, предсказанного dlib, 
        # в формат (x, y, w, h), который обычно используется в OpenCV
        x = rect.left()
        y = rect.top()
        w = rect.right()
        h = rect.bottom()

        # return a tuple of (x, y, w, h)
        return (x, y, w, h)

        return np.array(img)

# Окно приложения

In [19]:
class WindowApp:
    
    root = Tk()
    # путь к изображению, выбранного для идентификации 
    path_img_to_identify = ""
    
    # путь к изображению для нового человека (при добавлении человека)
    path_to_new_person = ""
    
    def __init__(self, handler_for_face, data_base, classificator):
        self.handler_for_face = handler_for_face
        self.data_base = data_base
        self.my_classif = classificator
    
    def create_ui(self):
        self.root.title("Ідентифікація людини")
        self.root.geometry("1000x600")
        self.root.resizable(width=False, height=False)
        self.root.configure(background='#c2d6d6')

        # =========== LEFT SIDE ==============
        self.frame_left = Frame(self.root, bg='gray')
        self.frame_left.place(relx=0.01, rely=0.05, relwidth=0.47, relheight=0.8)

        self.button_choose = Button(self.frame_left, text="Обрати зображення", command=self.show_image, background="#527a7a", font="Arial 14")
        self.button_choose.place(relx=0.1, rely=0.85, relwidth=0.3, relheight=0.07)

        self.photo_left = Label(self.frame_left)
        self.photo_left.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.7)

        self.text_id_photo = Label(self.frame_left, text="Зображення для ідентифікації", font="Arial 16", bg='gray', fg='white')
        self.text_id_photo.pack(side=tk.TOP)

        # =========== RIGHT SIDE ==============
        self.frame_right = Frame(self.root, bg='gray')
        self.frame_right.place(relx=0.51, rely=0.05, relwidth=0.48, relheight=0.8)

        self.photo_right = Label(self.frame_right)
        self.photo_right.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.7)

        self.text_right_photo = Label(self.frame_right, text="Знайдена людина у БД", font="Arial 16", bg='gray', fg='white')
        self.text_right_photo.pack(side=tk.TOP)

        self.text_name_person = Label(self.frame_right, text="Им'я: ", font="Arial 16", bg='gray', fg='white')
        self.text_name_person.place(relx=0.05, rely=0.85)

        self.button_add_person = Button(self.frame_right, text="Додати людину до БД", command=self.add_person_to_db, background="#527a7a", font="Arial 14")
        self.button_add_person.place(relx=0.58, rely=0.92, relwidth=0.4, relheight=0.07)

        # =========== BOTTOM SIDE ==============
        self.button_recognize = Button(self.root, text="Ідентифікувати", command=self.recognize_person, background="#527a7a", font="Arial 16")
        self.button_recognize.place(relx=0.35, rely=0.9, relwidth=0.3, relheight=0.08)

    # Метод-обработчик для выбора фото при нажатии кнопки "Обрати фото"
    def show_image(self):
        
        fln = filedialog.askopenfilename(initialdir=os.getcwd(), title="Вибір фото",
                                        filetypes=(("JPG File", "*.jpg"),
                                                   ("JPEG File", "*.jpeg"),
                                                   ("PNG File", "*.png"),
                                                   ("All files", "*.*")))

        img = Image.open(fln)
        img.thumbnail((400,400))
        img = ImageTk.PhotoImage(img)
        self.photo_left.configure(image=img)
        self.photo_left.image = img
        self.path_img_to_identify = fln
    
    # Метод-обработчик при нажатии кнопки "Ідентифікувати"
    def recognize_person(self):
    
        flag_found = False

        img_to_recognize = Utilities.load_image(self.path_img_to_identify)
        detected_faces = self.handler_for_face.detector(img_to_recognize, 1) # определение области лица HOG

        if len(detected_faces) == 0:
            print("Error: There is no faces!")
            return

        # извлечение 128 вектора из фото для идентификации
        encoded_face = face_handler.detect_face_and_encode(self.path_img_to_identify, "", image_identify=True)

        # KNN predict
        check = self.my_classif.classifier.predict_proba([encoded_face])
        print(check)

        box_face = Utilities.rect_to_box(detected_faces[0])

        name_of_person = "Особи немає у базі даних"

        # =========== Использование KNN ===========
        value_predict = np.amax(check)
        value_index = check.argmax()
        if value_predict > 0.6:
            flag_found = True
            name_of_person = Utilities.get_unique_values(self.data_base.known_face_names)[value_index]

        # =========================================

        cv2.rectangle(img_to_recognize, (box_face[0], box_face[1]), (box_face[2], box_face[3]), (0, 255, 0), 2)
        font = cv2.FONT_HERSHEY_DUPLEX
        #cv2.putText(img_to_recognize, name_of_person, (box_face[0], box_face[3]), font, 0.5, (255, 255, 255), 1)

        # ============ ОТОБРАЖЕНИЕ В ОКНЕ ПРИЛОЖЕНИЯ ===============
        # отрисовка слева фото с выделенным лицом
        img_recognize = Image.fromarray(img_to_recognize)
        img_recognize.thumbnail((400,400))
        img_recognize = ImageTk.PhotoImage(img_recognize)
        self.photo_left.configure(image=img_recognize)
        self.photo_left.image = img_recognize

        # Если человек найден - отобразить его фото из БД
        if flag_found == True:
            img_in_db = Image.open("images_db/" + name_of_person + "/" + name_of_person + ".jpg")
            img_in_db.thumbnail((400,400))
            img_in_db = ImageTk.PhotoImage(img_in_db)
            self.photo_right.configure(image=img_in_db)
            self.photo_right.image = img_in_db
        else: # Иначе отобразить фото Инкогнито
            unknown_img = Image.open("Unknown_person.png")
            unknown_img.thumbnail((400,400))
            unknown_img = ImageTk.PhotoImage(unknown_img)
            self.photo_right.configure(image=unknown_img)
            self.photo_right.image = unknown_img

        self.text_name_person.config(text="Имя: " + name_of_person)
        
    # ============== ОКНО ДОБАВЛЕНИЯ ЧЕЛОВЕКА =============
    
    # Метод-обработчик для кнпоки "Додати людину до БД"    
    def add_person_to_db(self):

        new_window = tk.Toplevel(self.root)

        new_window.title("Додавання людини до БД")
        new_window.geometry("480x240")
        new_window.resizable(width=False, height=False)
        new_window.configure(background='#a3c2c2')

        self.name_choosed_img = Label(new_window, text="Зображення не обране!", font="Arial 20", bg='#a3c2c2', fg='#ffffff', borderwidth=2)
        self.name_choosed_img.place(relx=0.05, rely=0.2)

        button_choose_photo = tk.Button(new_window, text = "Обрати фото", command=self.choose_person_img)
        button_choose_photo.place(relx=0.75, rely=0.2, relwidth=0.2, relheight=0.15)

        text_input_name = Label(new_window, text="Введіть им'я: ", font="Arial 20", bg='#a3c2c2', fg='#ffffff', borderwidth=2)
        text_input_name.place(relx=0.10, rely=0.5)

        entry_field = Entry(new_window)
        entry_field.place(relx=0.40, rely=0.5, relwidth=0.5)

        def add_func():
            new_dir_name = "images_db/" + entry_field.get()
            # создание новой папки для человека
            os.mkdir(new_dir_name)
            # копирование в папку выбранной фотографии
            shutil.copyfile(self.path_to_new_person, new_dir_name + "/" + entry_field.get() + ".jpg")

            new_path_to_image = new_dir_name + "/" + entry_field.get() + ".jpg"
            # выполнение аугментации
            Augmentator.augmentate_image(new_path_to_image)

            # получить 128-признаки из аугментированных фото
            self.process_person_img(new_dir_name, entry_field.get())

            new_window.destroy()

        button_add = Button(new_window, text = "Додати", command=add_func)
        button_add.place(relx=0.4, rely=0.8, relwidth=0.2, relheight=0.15)

    # открытие проводника для выбора фото
    def choose_person_img(self):

        fln = filedialog.askopenfilename(initialdir=os.getcwd(), title="Вибір фото",
                                        filetypes=(("JPG File", "*.jpg"),
                                                   ("JPEG File", "*.jpeg"),
                                                   ("PNG File", "*.png"),
                                                   ("All files", "*.*")))
        self.path_to_new_person = fln


        img_name = os.path.split(self.path_to_new_person)[1]
        self.name_choosed_img.config(text="Зображення: " + img_name)

    
    def process_person_img(self, path_to_dir, name_person):
        # получение последнего номера класса
        id_new_person = self.data_base.data_labels[-1] + 1

        # проход по всем фотографиям нового человека
        for img_in_person_dir in os.listdir(path_to_dir):

            # получение закодированного лица
            encoded_face = self.handler_for_face.detect_face_and_encode(path_to_dir, img_in_person_dir)
            # добавление закодированного лица в БД
            if len(encoded_face) == 128:
                self.data_base.add_to_db(encoded_face, id_new_person, name_person)

        # сохранение обновленной БД в файл
        Serializer_DB.save_data_to_file(self.data_base.known_face_encodings, self.data_base.known_face_names, self.data_base.data_labels)
        # переобучение классификатора
        self.my_classif.train_classifier_KNN(self.data_base)
    

In [20]:
# ==== Точка запуска программы ====
main_window = WindowApp(face_handler, data_base, my_classifier)
main_window.create_ui()

main_window.root.mainloop()

[[0.33333333 0.         0.         0.         0.33333333 0.
  0.33333333 0.         0.         0.         0.        ]]
Writing DB to file...
Re-FIT KNN!
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.66666667
  0.33333333]]
[[0.33333333 0.         0.         0.         0.         0.
  0.         0.         0.33333333 0.33333333 0.         0.
  0.        ]]
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.66666667
  0.33333333]]
